In [1]:
import numpy as np
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import nltk
import spacy
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize, RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import string
import re
from sklearn.feature_extraction import text 
%matplotlib inline 

In [2]:
data = pd.read_csv("blogtext.csv")

In [3]:
data.columns


Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

In [4]:
data.shape

(681284, 7)

In [5]:
data['text']

0                    Info has been found (+/- 100 pages,...
1                    These are the team members:   Drewe...
2                    In het kader van kernfusie op aarde...
3                          testing!!!  testing!!!          
4                      Thanks to Yahoo!'s Toolbar I can ...
                                ...                        
681279           Dear Susan,  I could write some really ...
681280           Dear Susan,  'I have the second yeast i...
681281           Dear Susan,  Your 'boyfriend' is fuckin...
681282           Dear Susan:    Just to clarify, I am as...
681283           Hey everybody...and Susan,  You might a...
Name: text, Length: 681284, dtype: object

In [6]:
new_data = data.head(9000)

In [7]:
new_data['label'] = new_data[new_data.columns[1:5]].apply(lambda x: (','.join(x.dropna().astype(str))),axis=1)

C:\Users\pg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
new_data

,id,gender,age,topic,sign,date,text,label
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...","male,15,Student,Leo"
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,"male,15,Student,Leo"
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,"male,15,Student,Leo"
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,"male,15,Student,Leo"
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,"male,33,InvestmentBanking,Aquarius"
...,...,...,...,...,...,...,...,...
8995,3477296,male,15,Student,Virgo,"17,July,2004",wah! how cold was it today? *...,"male,15,Student,Virgo"
8996,3477296,male,15,Student,Virgo,"17,July,2004",Could someone give me some su...,"male,15,Student,Virgo"
8997,3477296,male,15,Student,Virgo,"17,July,2004",I am blogging here tonight on...,"male,15,Student,Virgo"
8998,3477296,male,15,Student,Virgo,"16,July,2004",'It isn't pollution that's ha...,"male,15,Student,Virgo"


In [9]:
new_data = new_data.drop(columns=['gender','age','topic','sign','id','date'])

In [10]:
new_data

,text,label
0,"Info has been found (+/- 100 pages,...","male,15,Student,Leo"
1,These are the team members: Drewe...,"male,15,Student,Leo"
2,In het kader van kernfusie op aarde...,"male,15,Student,Leo"
3,testing!!! testing!!!,"male,15,Student,Leo"
4,Thanks to Yahoo!'s Toolbar I can ...,"male,33,InvestmentBanking,Aquarius"
...,...,...
8995,wah! how cold was it today? *...,"male,15,Student,Virgo"
8996,Could someone give me some su...,"male,15,Student,Virgo"
8997,I am blogging here tonight on...,"male,15,Student,Virgo"
8998,'It isn't pollution that's ha...,"male,15,Student,Virgo"


In [11]:
print(len(new_data.text))

9000


In [12]:
new_data.text[0]

'           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         '

In [13]:
nlp = spacy.load('en_core_web_sm')
l1 = ('btw','zza','zzzexy','zzzzz','youuuuu')
nlp.Defaults.stop_words.add(l1)


In [14]:
#for i in range(len(new_data.text)):
 #   new_data.text[i] = new_data.text[i].lower()

In [15]:
new_data.text

0                  Info has been found (+/- 100 pages,...
1                  These are the team members:   Drewe...
2                  In het kader van kernfusie op aarde...
3                        testing!!!  testing!!!          
4                    Thanks to Yahoo!'s Toolbar I can ...
                              ...                        
8995                     wah! how cold was it today? *...
8996                     Could someone give me some su...
8997                     I am blogging here tonight on...
8998                     'It isn't pollution that's ha...
8999                     Seems like everyone these day...
Name: text, Length: 9000, dtype: object

In [16]:
new_data.dtypes

text     object
label    object
dtype: object

In [17]:
for i in range(len(new_data.text)):
    tokenizer = RegexpTokenizer(r'\w+')
    new_data.text[i] = new_data.text[i].lower()
    word_tokens = tokenizer.tokenize(new_data.text[i])
    filtered_sentence = [w for w in word_tokens if not w in stopwords.words('english')] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in (nlp.Defaults.stop_words or string.punctuation):
            #if not w.isalpha():
            filtered_sentence.append(re.sub(r"[^a-zA-Z0-9]+", ' ',w ))
   
    new_data.text[i] = " ".join(filtered_sentence)
#print(word_tokens) 
#    print(new_data.text_new[i])
%time

Wall time: 0 ns


In [18]:
new_data.text

0       info found 100 pages 4 5 mb pdf files wait unt...
1       team members drewes van der laag urllink mail ...
2       het kader van kernfusie op aarde maak je eigen...
3                                         testing testing
4       thanks yahoo s toolbar capture urls popups mea...
                              ...                        
8995    wah cold today brrr wake 7 worth grins said sl...
8996    suggestions music dls im open asian music look...
8997    blogging tonight request andy know blog tonigh...
8998    isn t pollution s harming environment s impuri...
8999    like days turning religion times peril terror ...
Name: text, Length: 9000, dtype: object

In [19]:
X = new_data.text
Y = new_data.label

In [20]:

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=524)

In [21]:
# define a function that accepts a vectorizer and calculates the accuracy
def tokenize_test(vect):
    X_train_dtm = vect.fit_transform(X_train)
    print('Features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    #feature = nb.feature_count_
    #print("NB feature shape", feature)
   # print(nb.feature_count_.shape)
    y_train_pred = nb.predict(X_train_dtm)
    y_pred_class = nb.predict(X_test_dtm)
    print('Train Accuracy for NB : ', metrics.accuracy_score(y_train,y_train_pred))
    print('Test Accuracy for NB: ', metrics.accuracy_score(y_test, y_pred_class))
    logreg = LogisticRegression(C=1e9)
    logreg.fit(X_train_dtm, y_train)
    y_pred_class_LR = logreg.predict(X_test_dtm)
#print(metrics.accuracy_score(y_test, y_pred_class))
    y_train_LR = logreg.predict(X_train_dtm)
    print('Train Accuracy for LR: ',metrics.accuracy_score(y_train, y_train_LR))
    print('Test Accuracy for LR: ',metrics.accuracy_score(y_test, y_pred_class_LR))
%time

Wall time: 0 ns


In [22]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

Features:  418363
Train Accuracy for NB :  0.7691851851851852
Test Accuracy for NB:  0.3671111111111111


C:\Users\pg\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Train Accuracy for LR:  0.9948148148148148
Test Accuracy for LR:  0.49777777777777776


In [23]:
X_train_dtm = vect.fit_transform(X_train)
print('Features: ', X_train_dtm.shape[1])
X_test_dtm = vect.transform(X_test)
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
    #feature = nb.feature_count_
    #print("NB feature shape", feature)
   # print(nb.feature_count_.shape)
y_train_pred = nb.predict(X_train_dtm)
y_pred_class = nb.predict(X_test_dtm)
print('Train Accuracy for NB : ', metrics.accuracy_score(y_train,y_train_pred))
print('Test Accuracy for NB: ', metrics.accuracy_score(y_test, y_pred_class))

Features:  418363
Train Accuracy for NB :  0.7691851851851852
Test Accuracy for NB:  0.3671111111111111


In [24]:
# features names
feature_names = vect.get_feature_names()
print(feature_names[50:100])

['00 newton', '00 night', '00 nospamyahoo', '00 ok', '00 ouch', '00 plus', '00 pm', '00 presentations', '00 rarely', '00 ravenwood', '00 recently', '00 said', '00 saw', '00 sec', '00 showing', '00 speeding', '00 store', '00 sumber', '00 supposed', '00 taken', '00 tcr1', '00 teh', '00 time', '00 tired', '00 today', '00 tomorrow', '00 truth', '00 unit', '00 usher', '00 ve', '00 week', '00 woke', '00 yahoo', '00 yes', '000', '000 00', '000 000', '000 100', '000 1998', '000 36', '000 acre', '000 active', '000 address', '000 albums', '000 amish', '000 animals', '000 automobile', '000 bad', '000 btu', '000 cds', '000 checks', '000 companion', '000 compared', '000 copies', '000 cops', '000 country', '000 dangerous', '000 dollar', '000 dollars', '000 drop', '000 end', '000 english', '000 excited', '000 fact', '000 families', '000 far', '000 feet', '000 ferrari', '000 fight', '000 figure', '000 financial', '000 fix', '000 flights', '000 fucking', '000 ground', '000 guess', '000 head', '000 hear

In [25]:
y_train.head(4)

2122    male,35,Technology,Aries
5676     female,27,indUnk,Taurus
7396       male,36,Fashion,Aries
1668    male,35,Technology,Aries
Name: label, dtype: object

In [26]:
len(y_train)

6750

In [27]:
d = {}

In [28]:
d = y_train.apply(lambda x : pd.value_counts(x.split(","))).sum(axis = 0).to_dict()

In [29]:
d

{'35': 1753.0,
 'Technology': 1970.0,
 'Aries': 3155.0,
 'male': 4160.0,
 '27': 665.0,
 'female': 2590.0,
 'Taurus': 588.0,
 'indUnk': 2051.0,
 '36': 1298.0,
 'Fashion': 1230.0,
 'Pisces': 288.0,
 'Internet': 89.0,
 'Aquarius': 301.0,
 '33': 102.0,
 'InvestmentBanking': 54.0,
 '24': 346.0,
 'Engineering': 102.0,
 'Libra': 372.0,
 'Scorpio': 672.0,
 '17': 745.0,
 'Education': 184.0,
 '34': 414.0,
 'Student': 689.0,
 '14': 133.0,
 '25': 270.0,
 'Sagittarius': 664.0,
 'Cancer': 230.0,
 'Capricorn': 101.0,
 'Gemini': 107.0,
 '15': 416.0,
 '26': 143.0,
 'Leo': 163.0,
 '23': 156.0,
 'Marketing': 9.0,
 'Banking': 14.0,
 'BusinessServices': 62.0,
 'Sports-Recreation': 53.0,
 '16': 126.0,
 'Religion': 8.0,
 '41': 15.0,
 '13': 29.0,
 'Communications-Media': 76.0,
 '39': 60.0,
 'Virgo': 109.0,
 'Consulting': 10.0,
 'Law': 6.0,
 'Science': 45.0,
 '42': 9.0,
 'Automotive': 7.0,
 'Arts': 26.0,
 '38': 37.0,
 'Non-Profit': 57.0,
 '37': 13.0,
 '43': 4.0,
 'Accounting': 4.0,
 '45': 10.0,
 '44': 2.0,
 'M

In [30]:
from sklearn.preprocessing import MultiLabelBinarizer

In [31]:
mlb = MultiLabelBinarizer()

In [32]:
y_train_mlb = mlb.fit_transform(y_train)

In [33]:
y_test_mlb = mlb.transform(y_test)

In [34]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [35]:
LR = LogisticRegression(solver = 'lbfgs',random_state= 111)

In [36]:
LR

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=111, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [37]:
clf = OneVsRestClassifier(LR)

In [38]:
names = vect.get_feature_names()

In [39]:
y_train_mlb

array([[1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [40]:
clf.fit(X_train_dtm,y_train_mlb)
%time

C:\Users\pg\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label 0 is present in all training examples.
  str(classes[c]))
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\linear_model\

Wall time: 0 ns


In [41]:
y_pred_clf = clf.predict(X_test_dtm)

In [42]:
print(metrics.accuracy_score(y_test_mlb,y_pred_clf))

0.2653333333333333


In [43]:
from sklearn.metrics import classification_report

In [44]:
print(classification_report(y_test_mlb, y_pred_clf))

C:\Users\pg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2250
           1       0.71      0.19      0.30        64
           2       0.00      0.00      0.00         0
           3       0.86      0.46      0.60       542
           4       0.76      0.35      0.48       526
           5       0.80      0.83      0.81      1241
           6       0.88      0.36      0.51       331
           7       0.76      0.55      0.64       797
           8       0.89      0.48      0.62       498
           9       0.83      0.33      0.47       505
          10       0.75      0.33      0.46         9
          11       1.00      0.11      0.19        19
          12       0.79      0.78      0.78      1129
          13       0.89      0.19      0.31        43
          14       0.83      0.16      0.27       158
          15       0.76      0.15      0.25        87
          16       0.95      0.54      0.69       392
          17       1.00    

In [45]:
metrics.average_precision_score(y_test_mlb, y_pred_clf,average='micro')

0.7864627878473044

recall score for  average type Micro

In [47]:
#metrics.recall_score(y_test_mlb, y_pred_clf)
metrics.recall_score(y_test_mlb, y_pred_clf, labels=None, pos_label=1, average='micro', sample_weight=None)

0.7916797160455162

recall score for  average type Macro

In [64]:

metrics.recall_score(y_test_mlb, y_pred_clf, labels=None, pos_label=1, average='macro', sample_weight=None)

C:\Users\pg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.4801094213369808

recall score for  average type weighted

In [67]:

metrics.recall_score(y_test_mlb, y_pred_clf, labels=None, pos_label=1, average='weighted', sample_weight=None)

0.7916797160455162

In [68]:
metrics.multilabel_confusion_matrix(y_test_mlb, y_pred_clf,sample_weight=None,labels=None,samplewise=False)

array([[[   0,    0],
        [   0, 2250]],

       [[2181,    5],
        [  52,   12]],

       [[2250,    0],
        [   0,    0]],

       [[1668,   40],
        [ 292,  250]],

       [[1664,   60],
        [ 341,  185]],

       [[ 749,  260],
        [ 210, 1031]],

       [[1903,   16],
        [ 212,  119]],

       [[1313,  140],
        [ 356,  441]],

       [[1723,   29],
        [ 259,  239]],

       [[1712,   33],
        [ 340,  165]],

       [[2240,    1],
        [   6,    3]],

       [[2231,    0],
        [  17,    2]],

       [[ 881,  240],
        [ 249,  880]],

       [[2206,    1],
        [  35,    8]],

       [[2087,    5],
        [ 133,   25]],

       [[2159,    4],
        [  74,   13]],

       [[1846,   12],
        [ 180,  212]],

       [[2217,    0],
        [  31,    2]],

       [[2199,    6],
        [  33,   12]],

       [[2044,   20],
        [ 146,   40]],

       [[2226,    0],
        [  23,    1]],

       [[2236,    0],
        [  1

Precision, recall and F1 score for Micro average

In [54]:
metrics.precision_recall_fscore_support(y_test_mlb, y_pred_clf, beta=1.0, average='micro', warn_for=('precision', 'recall', 'f-score'), sample_weight=None)

(0.9055465997970028, 0.7916797160455162, 0.8447934943047317, None)

Precision, recall and F1 score for Macro average

In [62]:
metrics.precision_recall_fscore_support(y_test_mlb, y_pred_clf, beta=1.0, average='macro', warn_for=('precision', 'recall', 'f-score'), sample_weight=None)

C:\Users\pg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(0.8046078147741718, 0.4801094213369808, 0.55133580650704, None)

Precision, recall and F1 score for Weighted average

In [63]:
metrics.precision_recall_fscore_support(y_test_mlb, y_pred_clf, beta=1.0, average='weighted', warn_for=('precision', 'recall', 'f-score'), sample_weight=None)

(0.8928966542891479, 0.7916797160455162, 0.8233007112390947, None)

In [48]:
y_pred_clf[10:15]

array([[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
        1, 1, 1, 1, 0, 0, 0]])

In [49]:
y_test_mlb[10:15]

array([[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 0, 0]])